In [4]:
import pandas as pd
import glob
import os
from folium.plugins import HeatMap, MarkerCluster
import folium

In [2]:
clean_df = pd.read_csv("cleaned_df.csv")
clean_df.head(5)


,Unnamed: 0,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,YEAR,Lat,Long
0,0,HARASSMENT,C11,397,2015,42.291093,-71.065945
1,2,PROPERTY - LOST,B3,433,2015,42.283634,-71.082813
2,3,INVESTIGATE PERSON,A7,20,2015,42.377023,-71.032247
3,4,THREATS TO DO BODILY HARM,C11,359,2015,42.293606,-71.071887
4,5,PROPERTY - LOST,B2,282,2015,42.328663,-71.085634


In [3]:
print(clean_df['YEAR'].unique())
print(clean_df['OFFENSE_DESCRIPTION'].value_counts())

[2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]
OFFENSE_DESCRIPTION
INVESTIGATE PERSON                               54058
M/V - LEAVING SCENE - PROPERTY DAMAGE            38621
SICK/INJURED/MEDICAL - PERSON                    32395
VANDALISM                                        30717
INVESTIGATE PROPERTY                             29440
                                                 ...  
LARCENY BICYCLE $200 & OVER                          1
FRAUD - FALSE PRETENSE                               1
Migrated Report - Embezzlement                       1
Migrated Report - Injured/Medical/Sick Assist        1
Migrated Report - Kidnapping                         1
Name: count, Length: 273, dtype: int64


In [9]:
import folium
import pandas as pd
from folium.plugins import HeatMap, MarkerCluster

def create_crime_map_by_year(df, year=2024):
    """
    Create an interactive map visualization of crimes for a specific year using Folium
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns OFFENSE_DESCRIPTION, DISTRICT, 
                         REPORTING_AREA, YEAR, Lat, Long
    year (int): Year to filter the data (default: 2023)
    
    Returns:
    folium.Map: Interactive map object
    """
    # Filter data for specified year
    df_year = df[df['YEAR'] == year].copy()
    
    # Calculate the center of the map (mean of coordinates)
    center_lat = df_year['Lat'].mean()
    center_long = df_year['Long'].mean()
    
    # Create base map
    m = folium.Map(location=[center_lat, center_long], 
                  zoom_start=12,
                  tiles='OpenStreetMap')
    
    # Add title
    title_html = f'''
        <h3 align="center" style="font-size:16px">
            <b>Crime Incidents in {year}</b>
        </h3>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add marker cluster
    marker_cluster = MarkerCluster(name='Individual Crimes').add_to(m)
    
    # Create a dictionary to store crime counts by district
    # district_stats = df_year['DISTRICT'].value_counts().to_dict()
    
    # Add markers for each crime
    for idx, row in df_year.iterrows():
        
        # Add marker
        folium.Marker(
            location=[row['Lat'], row['Long']],
            tooltip=row['OFFENSE_DESCRIPTION'],
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(marker_cluster)
    
    # Add heatmap layer
    heat_data = df_year[['Lat', 'Long']].values.tolist()
    HeatMap(heat_data, name='Heat Map').add_to(m)
    
    # Add summary info
    summary_html = f'''
        <div style="position: fixed; 
                    bottom: 50px; left: 50px; width: 200px;
                    background-color: white; 
                    padding: 10px; 
                    border: 2px solid grey;
                    border-radius: 5px;
                    z-index: 1000;">
            <h4>Summary Statistics</h4>
            <p>Total Incidents: {len(df_year)}</p>
        </div>
    '''
    m.get_root().html.add_child(folium.Element(summary_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m, df_year

# Example usage:

# Assuming your DataFrame is called 'crime_df':
crime_map_2023, df_2023 = create_crime_map_by_year(clean_df, year=2024)
crime_map_2023.save('crime_map_2024.html')